# Foundry Agent Service and Knowledge Base Integration

## 📋 Overview

This notebook demonstrates how to integrate **Microsoft Foundry Agent Service** with **Azure AI Search Knowledge Base** through **MCP (Model Context Protocol)** for intelligent knowledge retrieval by Agents.

### 🔗 Integration Architecture

```mermaid
flowchart LR
    subgraph Foundry["Microsoft Foundry"]
        Agent["🤖 Agent"]
        Conn["🔗 Project Connection<br/>(RemoteTool)"]
    end

    subgraph Search["Azure AI Search"]
        MCP["📡 MCP Endpoint"]
        KB["📚 Knowledge Base"]
        KS["📂 Knowledge Source"]
        IDX[(Search Index)]
    end

    Agent -->|"Call MCP Tool"| Conn
    Conn -->|"ProjectManagedIdentity"| MCP
    MCP --> KB
    KB --> KS
    KS --> IDX
```

### 🔄 Call Flow

```mermaid
sequenceDiagram
    participant User as 👤 User
    participant Agent as 🤖 Agent
    participant MCP as 📡 MCP Endpoint
    participant KB as 📚 Knowledge Base
    
    User->>Agent: Ask question
    Agent->>MCP: knowledge_base_retrieve
    MCP->>KB: Retrieval request
    KB-->>MCP: Return chunks + references
    MCP-->>Agent: Return retrieval results
    Agent-->>User: Generate answer + citations
```

### ⚙️ Recommended Configuration

| Setting | Recommended Value | Description |
|---------|------------------|-------------|
| Output Mode | `EXTRACTIVE_DATA` | Return raw content, let Agent synthesize answer |
| Reasoning Effort | `minimal` | Skip LLM query planning, reduce latency and cost |
| Authentication | `ProjectManagedIdentity` | Use project managed identity, no key management needed |

---

## 📑 Table of Contents

| Step | Content | Description |
|------|---------|-------------|
| [Step 1](#step-1-configure-environment-variables) | Configure Environment Variables | Set up Search, Foundry Project, OpenAI connections |
| [Step 2](#step-2-create-search-index-and-knowledge-base) | Create Knowledge Base | Index → Knowledge Source → Knowledge Base |
| [Step 3](#step-3-create-project-connection-mcp-connection) | Create Project Connection | Use Management API to create RemoteTool connection |
| [Step 4](#step-4-create-agent-with-azure-ai-projects-sdk) | Create Agent | Configure Agent with MCPTool |
| [Step 5](#step-5-chat-with-agent) | Chat with Agent | Use Conversations API for Q&A |
| [Step 6](#step-6-classic-agent-api-alternative) | Classic Agent API | Use traditional Assistants-style API |
| [Step 7](#step-7-clean-up-resources-optional) | Clean Up Resources | Delete created resources |

---

## 📚 Reference Documentation

- [Connect Foundry IQ to Foundry Agent Service](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/knowledge-retrieval)
- [Build an end-to-end agentic retrieval solution](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-how-to-create-pipeline)
- [MCP Tool in Foundry Agents](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/mcp-tools)
- [GitHub Sample Code](https://github.com/Azure-Samples/azure-search-python-samples/tree/main/agentic-retrieval-pipeline-example)

In [ ]:
# Install required packages
%pip install azure-ai-projects==2.0.0b1 azure-identity azure-search-documents==11.7.0b2 python-dotenv requests -qU

In [ ]:
import os
import requests
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.mgmt.core.tools import parse_resource_id

# Load environment variables
load_dotenv()

print("✅ Packages imported successfully")

## Step 1: Configure Environment Variables

The following environment variables need to be configured (can be set in `.env` file):

```bash
# Azure AI Search
AZURE_SEARCH_ENDPOINT=https://your-search.search.windows.net

# Microsoft Foundry Project
PROJECT_ENDPOINT=https://your-resource.services.ai.azure.com/api/projects/your-project
PROJECT_RESOURCE_ID=/subscriptions/{sub-id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{account}/projects/{project}

# Azure OpenAI (via Foundry)
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com
AZURE_OPENAI_EMBEDDING_DEPLOYMENT=text-embedding-3-large
AGENT_MODEL=gpt-4o-mini
```

In [ ]:
# ============================================
# Configuration Parameters
# ============================================

# Azure AI Search Configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")

# Microsoft Foundry Project Configuration
project_endpoint = os.getenv("PROJECT_ENDPOINT")  # e.g., https://your-resource.services.ai.azure.com/api/projects/your-project
project_resource_id = os.getenv("PROJECT_RESOURCE_ID")  # Azure Resource ID

# Model Configuration
agent_model = os.getenv("AGENT_MODEL", "gpt-4o-mini")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# Resource Naming
index_name = os.getenv("AZURE_SEARCH_INDEX", "foundry-agent-demo-index")
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "foundry-agent-knowledge-source")
knowledge_base_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_BASE_NAME", "foundry-agent-knowledge-base")
project_connection_name = os.getenv("PROJECT_CONNECTION_NAME", "kb-mcp-connection")
agent_name = os.getenv("AGENT_NAME", "knowledge-retrieval-agent")

# Initialize credentials (use Azure CLI login)
credential = DefaultAzureCredential()

# Parse project resource ID
if project_resource_id:
    parsed_resource_id = parse_resource_id(project_resource_id)
    subscription_id = parsed_resource_id.get('subscription')
    resource_group = parsed_resource_id.get('resource_group')
    account_name = parsed_resource_id.get('name')
    project_name = parsed_resource_id.get('child_name_1')
else:
    print("⚠️ PROJECT_RESOURCE_ID not set")

# Verify configuration
print("📋 Configuration Check:")
print(f"   Search Endpoint: {search_endpoint}")
print(f"   Project Endpoint: {project_endpoint}")
print(f"   Agent Model: {agent_model}")
print(f"   Knowledge Base: {knowledge_base_name}")
print(f"   Agent Name: {agent_name}")

## Step 2: Create Search Index and Knowledge Base

First, create the index, knowledge source, and knowledge base in Azure AI Search.

**Note**: If you have already created these resources following `01_foundry_iq_knowledge_base.ipynb`, you can skip this step.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField, VectorSearch, VectorSearchProfile,
    HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters,
    SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField,
    SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference,
    KnowledgeBase, KnowledgeSourceReference,
    KnowledgeRetrievalOutputMode, KnowledgeRetrievalMinimalReasoningEffort
)

# Create index client
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

# Create index
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="content", type="Edm.String", searchable=True),
        SearchField(name="content_vector", type="Collection(Edm.Single)", stored=False,
                    vector_search_dimensions=3072, vector_search_profile_name="vector_profile"),
        SearchField(name="title", type="Edm.String", searchable=True, filterable=True),
        SearchField(name="source", type="Edm.String", filterable=True),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="vector_profile", algorithm_configuration_name="hnsw_config",
                                       vectorizer_name="azure_openai_vectorizer")],
        algorithms=[HnswAlgorithmConfiguration(name="hnsw_config")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=embedding_deployment,
                    model_name=embedding_deployment
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(field_name="title"),
                    content_fields=[SemanticField(field_name="content")]
                )
            )
        ]
    )
)

index_client.create_or_update_index(index)
print(f"✅ Index '{index_name}' created successfully")

In [ ]:
# Upload sample documents
from azure.search.documents import SearchClient
from openai import AzureOpenAI

# Initialize OpenAI client
aoai_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_version="2024-06-01",
    azure_ad_token_provider=get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
)

# Sample documents
documents = [
    {
        "id": "1",
        "title": "Foundry Agent Service Overview",
        "content": "Microsoft Foundry Agent Service allows you to create AI Agents customized with custom instructions and advanced tools like code interpreter and custom functions. Agents can use MCP (Model Context Protocol) tools to connect to Knowledge Bases for knowledge retrieval.",
        "source": "foundry-docs",
        "page_number": 1
    },
    {
        "id": "2",
        "title": "MCP Tool Integration",
        "content": "MCP (Model Context Protocol) is the standard protocol for Agent to Knowledge Base communication. By creating a RemoteTool project connection, Agents can securely call the knowledge_base_retrieve tool to retrieve relevant content and generate responses with citations.",
        "source": "foundry-docs",
        "page_number": 2
    },
    {
        "id": "3",
        "title": "Agent Creation Best Practices",
        "content": "Best practices for creating Agents: 1) Use clear instructions telling the Agent it must use the knowledge base to answer questions; 2) Require the Agent to provide citations in answers; 3) If the answer cannot be found, respond rather than guess.",
        "source": "foundry-docs",
        "page_number": 3
    }
]

# Generate embeddings
print("🔄 Generating vector embeddings...")
for doc in documents:
    response = aoai_client.embeddings.create(input=doc["content"], model=embedding_deployment)
    doc["content_vector"] = response.data[0].embedding
    print(f"   ✓ {doc['title']}")

# Upload documents
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents=documents)
print(f"\n✅ Uploaded {len(documents)} documents successfully")

In [ ]:
# Create Knowledge Source
knowledge_source = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Foundry Agent Service related documentation",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_fields=[
            SearchIndexFieldReference(name="id"),
            SearchIndexFieldReference(name="title"),
            SearchIndexFieldReference(name="page_number")
        ]
    )
)

index_client.create_or_update_knowledge_source(knowledge_source=knowledge_source)
print(f"✅ Knowledge Source '{knowledge_source_name}' created successfully")

In [ ]:
# Create Knowledge Base
# Note: When integrating with Foundry Agent Service, EXTRACTIVE_DATA mode and minimal reasoning effort are recommended
knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    description="Knowledge base for Foundry Agent Service",
    knowledge_sources=[
        KnowledgeSourceReference(name=knowledge_source_name)
    ],
    # EXTRACTIVE_DATA: Return raw content, let Agent synthesize answer (recommended)
    # ANSWER_SYNTHESIS: Knowledge Base directly generates answer
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    # minimal: Skip LLM query planning, reduce cost and latency (recommended for Agent integration)
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)

index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"✅ Knowledge Base '{knowledge_base_name}' created successfully")

# Build MCP endpoint URL
mcp_endpoint = f"{search_endpoint}/knowledgebases/{knowledge_base_name}/mcp?api-version=2025-11-01-Preview"
print(f"\n📍 MCP Endpoint: {mcp_endpoint}")

## Step 3: Create Project Connection (MCP Connection)

Create a RemoteTool connection in the Microsoft Foundry project pointing to the Knowledge Base's MCP endpoint.

This connection uses the project's managed identity for authentication.

In [ ]:
# Create Project Connection
# Need to use Azure Management API

# Get Bearer Token for Management API
bearer_token_provider = get_bearer_token_provider(credential, "https://management.azure.com/.default")
headers = {
    "Authorization": f"Bearer {bearer_token_provider()}",
    "Content-Type": "application/json"
}

# Request body for creating connection
connection_payload = {
    "name": project_connection_name,
    "type": "Microsoft.MachineLearningServices/workspaces/connections",
    "properties": {
        "authType": "ProjectManagedIdentity",  # Use project managed identity
        "category": "RemoteTool",              # Remote tool type
        "target": mcp_endpoint,                # MCP endpoint URL
        "isSharedToAll": True,                 # Share with all users
        "audience": "https://search.azure.com/",  # Target service
        "metadata": {
            "ApiType": "Azure"
        }
    }
}

# Send create connection request
connection_url = f"https://management.azure.com{project_resource_id}/connections/{project_connection_name}?api-version=2025-10-01-preview"

response = requests.put(connection_url, headers=headers, json=connection_payload)

if response.status_code in [200, 201]:
    print(f"✅ Project Connection '{project_connection_name}' created successfully")
    print(f"   Target: {mcp_endpoint}")
else:
    print(f"⚠️ Failed to create connection: {response.status_code}")
    print(f"   {response.text}")

## Step 4: Create Agent with azure-ai-projects SDK

Use `AIProjectClient` to create an Agent with MCP tools.

The Agent will use the `knowledge_base_retrieve` MCP tool to query the Knowledge Base.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

# Initialize Project Client
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

# List existing Agents
print("📋 Existing Agents:")
existing_agents = list(project_client.agents.list())
for agent in existing_agents:
    print(f"   • {agent.name} (version: {agent.version})")

if not existing_agents:
    print("   (none)")

In [ ]:
# Define Agent instructions
# These instructions optimize Knowledge Base call accuracy and citation formatting
agent_instructions = """
You are a helpful assistant that must use the knowledge base to answer all the questions from user. 
You must never answer from your own knowledge under any circumstances.

Every answer must always provide annotations for using the MCP knowledge base tool and render them as: 【message_idx:search_idx†source_name】

If you cannot find the answer in the provided knowledge base you must respond with "I don't know".
"""

# Create MCP tool configuration
mcp_kb_tool = MCPTool(
    server_label="knowledge-base",           # Tool label
    server_url=mcp_endpoint,                  # MCP endpoint URL
    require_approval="never",                # No human approval required
    allowed_tools=["knowledge_base_retrieve"], # Allowed tools
    project_connection_id=project_connection_name  # Project connection name
)

# Create Agent
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=agent_instructions,
        tools=[mcp_kb_tool]
    )
)

print(f"✅ Agent '{agent.name}' created successfully")
print(f"   Version: {agent.version}")
print(f"   Model: {agent_model}")

## Step 5: Chat with Agent

Use the Conversations API to chat with the Agent.

The Agent will automatically call the Knowledge Base's MCP tool to retrieve relevant content.

In [ ]:
# Get OpenAI client (for Conversations API)
openai_client = project_client.get_openai_client()

# Create conversation
conversation = openai_client.conversations.create()
print(f"✅ Conversation created: {conversation.id}")

def chat_with_agent(question: str):
    """
    Send a question to the Agent and get a response
    """
    print(f"\n❓ Question: {question}")
    print("=" * 60)
    
    # Send request, tool_choice="required" ensures Agent uses Knowledge Base
    response = openai_client.responses.create(
        conversation=conversation.id,
        tool_choice="required",  # Force tool usage
        input=question,
        extra_body={
            "agent": {
                "name": agent.name,
                "type": "agent_reference"
            }
        }
    )
    
    print(f"\n📝 Answer:\n{response.output_text}")
    return response

In [ ]:
# Test query
response = chat_with_agent("Who is responsible for security training?")

In [ ]:
# View response details
print(f"Response ID: {response.id}")
print(f"Model: {response.model}")
print(f"Token usage: {response.usage}")

In [ ]:
### Option 2: Classic Agent API (using message threads)

In [ ]:
# Create message thread
thread = project_client.agents.threads.create()
print(f"✅ Thread created: {thread.id}")

# Send message
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What does our security guidelines say about security training?"
)
print(f"Message sent: {message.id}")

In [ ]:
# Run and wait for completion
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Run status: {run.status}")

In [ ]:
# Get Agent's response
messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content in msg.content:
            if content.type == "text":
                print(f"📝 Agent response:\n{content.text.value}")

## 🧹 Step 6: Cleanup

In [ ]:
# Delete Agent (optional)
# project_client.agents.delete_agent(agent.id)
# print("Agent deleted")

## 📝 Summary

This notebook demonstrated how to integrate Foundry Agent Service with a Knowledge Base:

1. **MCP Tool Configuration**: Configure Knowledge Base as an Agent tool via `mcp_tool`
2. **Two Invocation Patterns**:
   - **Conversations API**: Suitable for simple single-turn conversations
   - **Classic Agent API**: Suitable for complex multi-turn interactions
3. **Automatic Grounding**: Agent automatically calls Knowledge Base to get accurate answers

For more details, please refer to:
- [Foundry Agent Service Documentation](https://learn.microsoft.com/en-us/azure/ai-services/agents/)
- [MCP Protocol Specification](https://spec.modelcontextprotocol.io/)